# 함수

In [4]:
import polars as pl
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

def drop_columns_with_high_null(df, threshold=0.95):
    """
    Polars 데이터프레임의 각 열마다 널값의 비율을 계산하여 threshold 이상이면 해당 열을 제거합니다.

    Parameters:
    df (pl.DataFrame): 널값 비율을 검사할 데이터프레임
    threshold (float): 널값 비율의 임계값 (기본값은 0.95)

    Returns:
    pl.DataFrame: 지정된 비율 이상의 널값을 가진 열이 제거된 데이터프레임
    """
    # 각 열마다 널값의 비율 계산
    null_counts = df.null_count()
    total_rows = df.shape[0]

    # 널값 비율을 계산하고 임계값 이상의 널값 비율을 가진 열 선택
    columns_to_drop = [
        column for column in df.columns
        if null_counts[column][0] / total_rows > threshold
    ]

    # 해당 열들을 데이터프레임에서 제거
    df_cleaned = df.drop(columns_to_drop)

    return df_cleaned, columns_to_drop

## date타입 변환
# 'D'로 끝나는 모든 컬럼에 대해 변환 수행
def datetype_preprocessing(df):
    for col in df.columns:
        if col.endswith('D'):
            # Handle null values by filling them with an empty string or some default date string
            df = df.with_columns(
                pl.col(col).fill_null('').cast(pl.Utf8)
            )

            # Convert the string to date
            df = df.with_columns(
                pl.col(col).str.strptime(pl.Date, "%Y-%m-%d", strict=False)
            )

            # Convert the date to integer format YYYYMMDD
            df = df.with_columns(
                pl.col(col).cast(pl.Int32).fill_null(0).alias(col)
            )
    return df

## lavel encoding
def label_encoding(df):
    if isinstance(df, pl.DataFrame):
        # Convert Polars dataframe to Pandas dataframe
        df = df.to_pandas()
    for column in df.columns:
        # 판다스 데이터프레임에서 문자열 열 확인
        if df[column].dtype == 'object':
            le = LabelEncoder()
            # 라벨 인코딩 수행
            df[column] = le.fit_transform(df[column])
    return df


##null 값 선형회귀로 채우기

def linear_regression_imputation(data, target_column, reference_columns):
    """
    데이터셋의 특정 칼럼의 결측치를 선형 회귀로 채웁니다.

    Parameters:
        data (DataFrame): 결측치를 채울 데이터셋
        target_column (str): 결측치를 채울 대상 칼럼의 이름
        reference_columns (list): 결측치를 채울 때 참고할 칼럼들의 이름 리스트

    Returns:
        DataFrame: 결측치가 채워진 데이터셋
    """
    # 대상 칼럼과 참고할 칼럼들로 이루어진 subset 생성
    subset = data[[target_column] + reference_columns]

    # 결측치를 가진 행과 결측치가 아닌 행을 분리
    missing_data = subset[subset[target_column].isnull()]
    complete_data = subset.dropna(subset=[target_column])

    # 선형 회귀 모델 학습
    X = complete_data[reference_columns]
    y = complete_data[target_column]
    model = LinearRegression()
    model.fit(X, y)

    # 결측치를 예측하여 채움
    imputed_values = model.predict(missing_data[reference_columns])
    data.loc[data[target_column].isnull(), target_column] = imputed_values

    return data

# 사용 예시
# linear_regression_imputation(data, 'target_column', ['reference_column1', 'reference_column2'])

##null 값 처리 함수
def null_preprocessing(df):
    if isinstance(df, pl.DataFrame):
        # Convert Polars dataframe to Pandas dataframe
        df = df.to_pandas()
    non_null_columns = [col for col in df.columns if df[col].notnull().all()]
    null_columns = [col for col in df.columns if df[col].isnull().any()]

    for column in null_columns:
        print(column)
        df = linear_regression_imputation(df, column, non_null_columns)
    return df

def num_group_by(df):
    # 각 그룹의 평균 계산
    df_mean = df.groupby('case_id').mean()
    df_mean.drop(['num_group1','num_group2'])
    return df_mean

def find_all_null_columns(df):
    all_null_columns = []
    for col in df.columns:
        if df.select(pl.col(col).is_null().sum()).item() == df.height:
            all_null_columns.append(col)
    return all_null_columns

def test_train_col_equ(train, test):
    # 훈련 세트와 테스트 세트의 열 이름 가져오기
    train_columns = set(train.columns)
    test_columns = set(test.columns)

    # 테스트 세트에만 있는 열 찾기
    test_only_columns = test_columns - train_columns
    print(f"Columns only in test set: {test_only_columns}")
    test = test.drop(test_only_columns)

    return test




In [3]:
import polars as pl
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import re
import polars.selectors as cs


def train_static_depth0_preprocess(train_static):
    if isinstance(train_static, pd.DataFrame):
        # pandas DataFrame을 polars DataFrame으로 변환
        train_static = pl.DataFrame(train_static)

    #actualdpdtolerance_344P 삭제
    train_static = train_static.drop(['actualdpdtolerance_344P'])
    train_static = train_static.with_columns(
        pl.when(train_static['annuity_780A'].is_not_null())
        .then(train_static['annuity_780A'])
        .otherwise(train_static['annuitynextmonth_57A'])
        .alias('annuity')
    )
    train_static = train_static.drop(['annuity_780A'])
    train_static = train_static.drop(['annuitynextmonth_57A'])
    #applicationscnt
    train_static = train_static.drop(['applicationcnt_361L','applications30d_658L','applicationscnt_1086L','applicationscnt_629L','applicationscnt_867L'])
    train_static = train_static.drop(['avgdbddpdlast3m_4187120P'])
    #전화번호 관련 열 다 삭제
    # 'clientscnt'로 시작하는 모든 열을 찾습니다.
    columns_to_drop = [column for column in train_static.columns if column.startswith('clientscnt')]
    # 이 열들을 삭제합니다.
    train_static = train_static.drop(columns_to_drop)
    # 'commnoinclast6m_3546845L' 다 0임
    train_static = train_static.drop('commnoinclast6m_3546845L')
    train_static = train_static.drop(['currdebt_22A','currdebtcredtyperange_828A']) # 삭제
    train_static = train_static.drop('homephncnt_628L') # 삭제
    train_static = train_static.drop('maxdbddpdlast1m_3658939P') # 삭제
    train_static = train_static.drop('maxdbddpdtollast6m_4187119P') # 삭제
    train_static = train_static.drop(['maxdpdlast12m_727P','maxdpdlast3m_392P','maxdpdlast6m_474P','maxdpdlast9m_1059P','maxdpdlast24m_143P'])
    train_static = train_static.drop('mobilephncnt_593L')

    train_static = train_static.drop('interestrategrace_34L')
    train_static = train_static.drop('equalityempfrom_62L')
    train_static = train_static.drop('clientscnt_136L')
    train_static = train_static.drop('lastdependentsnum_448L')
    train_static = train_static.drop('lastotherinc_902A')
    train_static = train_static.drop('lastotherlnsexpense_631A')
    train_static = train_static.drop('lastrepayingdate_696D')
    train_static = train_static.drop('maxannuity_4075009A')
    train_static = train_static.drop('validfrom_1069D')

    train_static = train_static.with_columns(
        pl.when(train_static['sumoutstandtotal_3546847A'].is_not_null())
        .then(train_static['sumoutstandtotal_3546847A'])
        .otherwise(train_static['sumoutstandtotalest_4493215A'])
        .alias('sumoutstandtotal')
    )
    train_static = train_static.drop('sumoutstandtotal_3546847A') # 삭제
    train_static = train_static.drop('sumoutstandtotalest_4493215A') # 삭제

    train_static, columns_to_drop = drop_columns_with_high_null(train_static,0.95)
    train_static = datetype_preprocessing(train_static)
    train_static = label_encoding(train_static)
    train_static = null_preprocessing(train_static)

    if isinstance(train_static, pd.DataFrame):
        # pandas DataFrame을 polars DataFrame으로 변환
        train_static = pl.DataFrame(train_static)

    return train_static, columns_to_drop



def test_static_depth0_preprocess(test_static, columns_to_drop_in_train):
    if isinstance(test_static, pd.DataFrame):
        # pandas DataFrame을 polars DataFrame으로 변환
        test_static = pl.DataFrame(test_static)

    #actualdpdtolerance_344P 삭제
    test_static = test_static.drop(['actualdpdtolerance_344P'])
    test_static = test_static.with_columns(
        pl.when(test_static['annuity_780A'].is_not_null())
        .then(test_static['annuity_780A'])
        .otherwise(test_static['annuitynextmonth_57A'])
        .alias('annuity')
    )
    test_static = test_static.drop(['annuity_780A'])
    test_static = test_static.drop(['annuitynextmonth_57A'])
    #applicationscnt
    test_static = test_static.drop(['applicationcnt_361L','applications30d_658L','applicationscnt_1086L','applicationscnt_629L','applicationscnt_867L'])
    test_static = test_static.drop(['avgdbddpdlast3m_4187120P'])
    #전화번호 관련 열 다 삭제
    # 'clientscnt'로 시작하는 모든 열을 찾습니다.
    columns_to_drop = [column for column in test_static.columns if column.startswith('clientscnt')]
    # 이 열들을 삭제합니다.
    test_static = test_static.drop(columns_to_drop)
    # 'commnoinclast6m_3546845L' 다 0임
    test_static = test_static.drop('commnoinclast6m_3546845L')
    test_static = test_static.drop(['currdebt_22A','currdebtcredtyperange_828A']) # 삭제
    test_static = test_static.drop('homephncnt_628L') # 삭제
    test_static = test_static.drop('maxdbddpdlast1m_3658939P') # 삭제
    test_static = test_static.drop('maxdbddpdtollast6m_4187119P') # 삭제
    test_static = test_static.drop(['maxdpdlast12m_727P','maxdpdlast3m_392P','maxdpdlast6m_474P','maxdpdlast9m_1059P','maxdpdlast24m_143P'])
    test_static = test_static.drop('mobilephncnt_593L')

    test_static = test_static.drop('interestrategrace_34L')
    test_static = test_static.drop('equalityempfrom_62L')
    test_static = test_static.drop('clientscnt_136L')
    test_static = test_static.drop('lastdependentsnum_448L')
    test_static = test_static.drop('lastotherinc_902A')
    test_static = test_static.drop('lastotherlnsexpense_631A')
    test_static = test_static.drop('lastrepayingdate_696D')
    test_static = test_static.drop('maxannuity_4075009A')
    test_static = test_static.drop('validfrom_1069D')


    test_static = test_static.with_columns(
        pl.when(test_static['sumoutstandtotal_3546847A'].is_not_null())
        .then(test_static['sumoutstandtotal_3546847A'])
        .otherwise(test_static['sumoutstandtotalest_4493215A'])
        .alias('sumoutstandtotal')
    )
    test_static = test_static.drop('sumoutstandtotal_3546847A') # 삭제
    test_static = test_static.drop('sumoutstandtotalest_4493215A') # 삭제

    test_static = test_static.drop(columns_to_drop_in_train)
    test_static = datetype_preprocessing(test_static)
    test_static = label_encoding(test_static)
    test_static = null_preprocessing(test_static)


    if isinstance(test_static, pd.DataFrame):
        # pandas DataFrame을 polars DataFrame으로 변환
        test_static = pl.DataFrame(test_static)

    return test_static


def train_credit_bureau_a_depth2_preprocess(train_credit_bureau_a_2):

    train_credit_bureau_a_2 = train_credit_bureau_a_2.with_columns(
        (pl.col('collater_valueofguarantee_1124L') > 0).cast(pl.Int8).alias('collater_valueofguarantee_1124L')
    )
    train_credit_bureau_a_2['collater_valueofguarantee_1124L'].fill_null(0)

    train_credit_bureau_a_2 = train_credit_bureau_a_2.with_columns(
        (pl.col('collater_valueofguarantee_876L') > 0).cast(pl.Int8).alias('collater_valueofguarantee_876L')
    )

    train_credit_bureau_a_2, columns_to_drop = drop_columns_with_high_null(train_credit_bureau_a_2,0.95)
    train_credit_bureau_a_2 = datetype_preprocessing(train_credit_bureau_a_2)
    train_credit_bureau_a_2 = label_encoding(train_credit_bureau_a_2)
    train_credit_bureau_a_2 = null_preprocessing(train_credit_bureau_a_2)

    if isinstance(train_credit_bureau_a_2, pd.DataFrame):
        # pandas DataFrame을 polars DataFrame으로 변환
        train_credit_bureau_a_2 = pl.DataFrame(train_credit_bureau_a_2)

    return train_credit_bureau_a_2, columns_to_drop

def test_credit_bureau_a_depth2_preprocess(test_credit_bureau_a_2, columns_to_drop_in_train):

    test_credit_bureau_a_2 = test_credit_bureau_a_2.with_columns(
        (pl.col('collater_valueofguarantee_1124L') > 0).cast(pl.Int8).alias('collater_valueofguarantee_1124L')
    )
    test_credit_bureau_a_2['collater_valueofguarantee_1124L'].fill_null(0)

    test_credit_bureau_a_2 = test_credit_bureau_a_2.with_columns(
        (pl.col('collater_valueofguarantee_876L') > 0).cast(pl.Int8).alias('collater_valueofguarantee_876L')
    )

    test_credit_bureau_a_2 = test_credit_bureau_a_2.drop(columns_to_drop_in_train)
    test_credit_bureau_a_2 = datetype_preprocessing(test_credit_bureau_a_2)
    test_credit_bureau_a_2 = label_encoding(test_credit_bureau_a_2)
    test_credit_bureau_a_2 = null_preprocessing(test_credit_bureau_a_2)


    if isinstance(test_credit_bureau_a_2, pd.DataFrame):
        # pandas DataFrame을 polars DataFrame으로 변환
        test_credit_bureau_a_2 = pl.DataFrame(test_credit_bureau_a_2)

    return test_credit_bureau_a_2


def train_credit_bureau_b_depth2_preprocess(train_credit_bureau_b_2):
    train_credit_bureau_b_2 = num_group_by(train_credit_bureau_b_2)
    train_credit_bureau_b_2 = train_credit_bureau_b_2.drop("pmts_date_1107D")
    train_credit_bureau_b_2, columns_to_drop = drop_columns_with_high_null(train_credit_bureau_b_2,0.95)
    train_credit_bureau_b_2 = datetype_preprocessing(train_credit_bureau_b_2)
    train_credit_bureau_b_2 = label_encoding(train_credit_bureau_b_2)
    train_credit_bureau_b_2 = null_preprocessing(train_credit_bureau_b_2)

    if isinstance(train_credit_bureau_b_2, pd.DataFrame):
        # pandas DataFrame을 polars DataFrame으로 변환
        train_credit_bureau_b_2 = pl.DataFrame(train_credit_bureau_b_2)

    return train_credit_bureau_b_2, columns_to_drop

def test_credit_bureau_b_depth2_preprocess(test_credit_bureau_b_2, columns_to_drop_in_train):
    test_credit_bureau_b_2 = num_group_by(test_credit_bureau_b_2)

    test_credit_bureau_b_2 = test_credit_bureau_b_2.drop("pmts_date_1107D")

    test_credit_bureau_b_2 = test_credit_bureau_b_2.drop(columns_to_drop_in_train)
    test_credit_bureau_b_2 = datetype_preprocessing(test_credit_bureau_b_2)
    test_credit_bureau_b_2 = label_encoding(test_credit_bureau_b_2)
    test_credit_bureau_b_2 = null_preprocessing(test_credit_bureau_b_2)

    if isinstance(test_credit_bureau_b_2, pd.DataFrame):
        # pandas DataFrame을 polars DataFrame으로 변환
        test_credit_bureau_b_2 = pl.DataFrame(test_credit_bureau_b_2)

    return test_credit_bureau_b_2


def train_credit_person_depth2_preprocess(train_credit_person_2):
    train_credit_person_2 = num_group_by(train_credit_person_2)
    train_credit_person_2, columns_to_drop = drop_columns_with_high_null(train_credit_person_2,0.95)
    train_credit_person_2 = datetype_preprocessing(train_credit_person_2)
    train_credit_person_2 = label_encoding(train_credit_person_2)
    train_credit_person_2 = null_preprocessing(train_credit_person_2)

    if isinstance(train_credit_person_2, pd.DataFrame):
        # pandas DataFrame을 polars DataFrame으로 변환
        train_credit_person_2 = pl.DataFrame(train_credit_person_2)

    return train_credit_person_2, columns_to_drop

def test_credit_person_depth2_preprocess(test_credit_person_2, columns_to_drop_in_train):
    test_credit_person_2 = num_group_by(test_credit_person_2)
    test_credit_person_2 = test_credit_person_2.drop(columns_to_drop_in_train)
    test_credit_person_2 = datetype_preprocessing(test_credit_person_2)
    test_credit_person_2 = label_encoding(test_credit_person_2)
    test_credit_person_2 = null_preprocessing(test_credit_person_2)

    if isinstance(test_credit_person_2, pd.DataFrame):
        # pandas DataFrame을 polars DataFrame으로 변환
        test_credit_person_2 = pl.DataFrame(test_credit_person_2)

    return test_credit_person_2

def train_credit_applprev_depth2_preprocess(train_credit_applprev_2):
    train_credit_applprev_2 = num_group_by(train_credit_applprev_2)
    train_credit_applprev_2, columns_to_drop = drop_columns_with_high_null(train_credit_applprev_2,0.95)
    train_credit_applprev_2 = datetype_preprocessing(train_credit_applprev_2)
    train_credit_applprev_2 = label_encoding(train_credit_applprev_2)
    train_credit_applprev_2 = null_preprocessing(train_credit_applprev_2)

    if isinstance(train_credit_applprev_2, pd.DataFrame):
        # pandas DataFrame을 polars DataFrame으로 변환
        train_credit_applprev_2 = pl.DataFrame(train_credit_applprev_2)

    return train_credit_applprev_2, columns_to_drop

def test_credit_applprev_depth2_preprocess(test_credit_applprev_2, columns_to_drop_in_train):
    test_credit_applprev_2 = num_group_by(test_credit_applprev_2)

    test_credit_applprev_2 = test_credit_applprev_2.drop(columns_to_drop_in_train)
    test_credit_applprev_2 = datetype_preprocessing(test_credit_applprev_2)
    test_credit_applprev_2 = label_encoding(test_credit_applprev_2)
    test_credit_applprev_2 = null_preprocessing(test_credit_applprev_2)

    if isinstance(test_credit_applprev_2, pd.DataFrame):
        # pandas DataFrame을 polars DataFrame으로 변환
        test_credit_applprev_2 = pl.DataFrame(test_credit_applprev_2)

    return test_credit_applprev_2



def train_static_cb_depth0_preprocess(train_static_cb):
    if isinstance(train_static_cb, pd.DataFrame):
        # pandas DataFrame을 polars DataFrame으로 변환
        train_static_cb = pl.DataFrame(train_static_cb)

    #assignment
    train_static_cb = train_static_cb.drop(['assignmentdate_238D','assignmentdate_4527235D','assignmentdate_4955616D'])



    #days 어차피 누적되는 데이터 이므로 days360빼고 다 삭제해준다.
    train_static_cb = train_static_cb.drop(['days30_165L','days90_310L','days120_123L','days180_256L'])


    #fail for3years가 'formonth_118L','forquarter_462L','forweek_601L','foryear_618L'이 데이터를 모두 포함함
    #'formonth_118L','forquarter_462L','forweek_601L','foryear_618L' drop
    train_static_cb = train_static_cb.drop(['formonth_118L','forquarter_462L','forweek_601L','foryear_618L'])


    # cancle for3years_584L가 'formonth _206L','forquarter_1017L','forweek_1077L','foryear_818L'이 데이터를 모두 포함함
    # 'formonth _206L','forquarter_1017L','forweek_1077L','foryear_818L' drop
    train_static_cb = train_static_cb.drop(['formonth_206L','forquarter_1017L','forweek_1077L','foryear_818L'])


    # credit_history for3years가 'formonth_535L','forquarter_634L','forweek_528L','foryear_850L'이 데이터를 모두 포함함
    # 'formonth_535L','forquarter_634L','forweek_528L','foryear_850L' drop
    train_static_cb = train_static_cb.drop(['formonth_535L','forquarter_634L','forweek_528L','foryear_850L'])



    #numberofqueries
    train_static_cb = train_static_cb.drop('numberofqueries_373L') # 삭제



    #4분기 실적만 남기기
    train_static_cb = train_static_cb.drop('firstquarter_103L') # 삭제
    train_static_cb = train_static_cb.drop('secondquarter_766L') # 삭제
    train_static_cb = train_static_cb.drop('thirdquarter_1082L') # 삭제
    train_static_cb = train_static_cb.with_columns(
        train_static_cb['fourthquarter_440L'].fill_null(0)
    )

    train_static_cb, columns_to_drop = drop_columns_with_high_null(train_static_cb,0.95)
    train_static_cb = datetype_preprocessing(train_static_cb)
    train_static_cb = label_encoding(train_static_cb)
    train_static_cb = null_preprocessing(train_static_cb)

    if isinstance(train_static_cb, pd.DataFrame):
        # pandas DataFrame을 polars DataFrame으로 변환
        train_static_cb = pl.DataFrame(train_static_cb)

    return train_static_cb, columns_to_drop

def test_static_cb_depth0_preprocess(test_static_cb, columns_to_drop_in_train):
    if isinstance(test_static_cb, pd.DataFrame):
        # pandas DataFrame을 polars DataFrame으로 변환
        test_static_cb = pl.DataFrame(test_static_cb)

     #assignment
    test_static_cb = test_static_cb.drop(['assignmentdate_238D','assignmentdate_4527235D','assignmentdate_4955616D'])



    #days 어차피 누적되는 데이터 이므로 days360빼고 다 삭제해준다.
    test_static_cb = test_static_cb.drop(['days30_165L','days90_310L','days120_123L','days180_256L'])


    #fail for3years가 'formonth_118L','forquarter_462L','forweek_601L','foryear_618L'이 데이터를 모두 포함함
    #'formonth_118L','forquarter_462L','forweek_601L','foryear_618L' drop
    test_static_cb = test_static_cb.drop(['formonth_118L','forquarter_462L','forweek_601L','foryear_618L'])


    # cancle for3years_584L가 'formonth _206L','forquarter_1017L','forweek_1077L','foryear_818L'이 데이터를 모두 포함함
    # 'formonth _206L','forquarter_1017L','forweek_1077L','foryear_818L' drop
    test_static_cb = test_static_cb.drop(['formonth_206L','forquarter_1017L','forweek_1077L','foryear_818L'])


    # credit_history for3years가 'formonth_535L','forquarter_634L','forweek_528L','foryear_850L'이 데이터를 모두 포함함
    # 'formonth_535L','forquarter_634L','forweek_528L','foryear_850L' drop
    test_static_cb = test_static_cb.drop(['formonth_535L','forquarter_634L','forweek_528L','foryear_850L'])



    #numberofqueries
    test_static_cb = test_static_cb.drop('numberofqueries_373L') # 삭제



    #4분기 실적만 남기기
    test_static_cb = test_static_cb.drop('firstquarter_103L') # 삭제
    test_static_cb = test_static_cb.drop('secondquarter_766L') # 삭제
    test_static_cb = test_static_cb.drop('thirdquarter_1082L') # 삭제
    test_static_cb = test_static_cb.with_columns(
        test_static_cb['fourthquarter_440L'].fill_null(0)
    )

    test_static_cb = test_static_cb.drop(columns_to_drop_in_train)
    test_static_cb = datetype_preprocessing(test_static_cb)
    test_static_cb = label_encoding(test_static_cb)
    test_static_cb = null_preprocessing(test_static_cb)

    if isinstance(test_static_cb, pd.DataFrame):
        # pandas DataFrame을 polars DataFrame으로 변환
        test_static_cb = pl.DataFrame(test_static_cb)

    return test_static_cb

In [ ]:
def datetype_preprocessing(df):
    for col in df.columns:
        if col.endswith('D'):
            # Fill null values with an empty string or a default date string
            df[col] = df[col].astype(str).fillna('')

            # Convert the string to datetime, errors='coerce' will turn invalid parsing into NaT
            df[col] = pd.to_datetime(df[col], format="%Y-%m-%d", errors='coerce')

            # Convert the date to integer format YYYYMMDD
            df[col] = df[col].dt.strftime('%Y%m%d').fillna('0').astype(int)

    return df

def column_description(df, column):
    feature_definitions = pd.read_csv(dataPath + 'feature_definitions.csv')
    definition = feature_definitions[feature_definitions['Variable'] == column]
    return definition

def calculate_missing_percentage(dataframe):
    """
    데이터프레임의 각 열에 대한 결측치 비율을 계산하고 오름차순으로 정렬하는 함수

    Parameters:
    dataframe (DataFrame): 결측치를 계산할 데이터프레임

    Returns:
    DataFrame: 각 열에 대한 결측치 비율이 포함된 데이터프레임
    """
    total_rows = len(dataframe)
    missing_count = dataframe.isnull().sum()
    missing_percentage = (missing_count / total_rows) * 100
    missing_data = pd.DataFrame({
        'Column': dataframe.columns,
        'Missing': missing_count,
        'Percent': missing_percentage
    }).reset_index(drop=True)

    # 결측치 비율에 따라 오름차순으로 정렬
    missing_data = missing_data.sort_values(by='Percent', ascending=True).reset_index(drop=True)

    return missing_data

def date_to_integer(df, column):
    df[column] = pd.to_datetime(df[column], errors='coerce')
    df[column+'int'] = df[column].dt.strftime('%Y%m%d').fillna('0').astype(int)
    df = df.drop(columns=[column])
    return df

def drop_columns(dataframe, threshold=80):
    """
    결측치 비율이 주어진 임계값을 초과하는 열을 삭제하는 함수

    Parameters:
    dataframe (DataFrame): 결측치를 확인할 데이터프레임
    threshold (float): 결측치 비율 임계값 (기본값: 80)

    Returns:
    DataFrame: 지정된 임계값 이상의 결측치를 가진 열이 삭제된 데이터프레임
    """
    missing_data = calculate_missing_percentage(dataframe)
    columns_to_drop = missing_data[missing_data['Percent'] > threshold]['Column']
    dataframe_dropped = dataframe.drop(columns=columns_to_drop)
    return dataframe_dropped

from sklearn.linear_model import LinearRegression

def linear_regression_imputation(data, target_column, reference_columns):
    """
    데이터셋의 특정 칼럼의 결측치를 선형 회귀로 채웁니다.

    Parameters:
        data (DataFrame): 결측치를 채울 데이터셋
        target_column (str): 결측치를 채울 대상 칼럼의 이름
        reference_columns (list): 결측치를 채울 때 참고할 칼럼들의 이름 리스트

    Returns:
        DataFrame: 결측치가 채워진 데이터셋
    """
    # 대상 칼럼과 참고할 칼럼들로 이루어진 subset 생성
    subset = data[[target_column] + reference_columns]

    # 결측치를 가진 행과 결측치가 아닌 행을 분리
    missing_data = subset[subset[target_column].isnull()]
    complete_data = subset.dropna(subset=[target_column])

    # 선형 회귀 모델 학습
    X = complete_data[reference_columns]
    y = complete_data[target_column]
    model = LinearRegression()
    model.fit(X, y)

    # 결측치를 예측하여 채움
    imputed_values = model.predict(missing_data[reference_columns])
    data.loc[data[target_column].isnull(), target_column] = imputed_values

    return data

# 사용 예시
# linear_regression_imputation(data, 'target_column', ['reference_column1', 'reference_column2'])

def columns_with_no_missing_values(df):
    """
    데이터프레임에서 결측치 개수가 0인 열의 이름을 반환하는 함수

    Parameters:
    df (DataFrame): 검사할 데이터프레임

    Returns:
    list: 결측치 개수가 0인 열의 이름을 담은 리스트
    """
    missing_data = calculate_missing_percentage(df)
    return missing_data[missing_data['Missing'] == 0].index.tolist()

from sklearn.preprocessing import LabelEncoder

def label_encode_column(data, column_name):
    """
    Label encode a column in the DataFrame.

    Parameters:
    - data: DataFrame containing the column to be label encoded.
    - column_name: Name of the column to be label encoded.

    Returns:
    - DataFrame with the specified column label encoded.
    """
    # Label encode the column
    label_encoder = LabelEncoder()
    data[column_name] = label_encoder.fit_transform(data[column_name])

    return data

from sklearn.preprocessing import LabelEncoder

def label_encoding(df):
    if isinstance(df, pl.DataFrame):
        # Convert Polars dataframe to Pandas dataframe
        df = df.to_pandas()
    elif not isinstance(df, pd.DataFrame):
        raise ValueError("Input must be a Pandas DataFrame or a Polars DataFrame")

    for column in df.columns:
        # Check if the column is of object type
        if df[column].dtype == 'object':
            le = LabelEncoder()
            # Perform label encoding
            df[column] = le.fit_transform(df[column])

    if isinstance(df, pl.DataFrame):
        # Convert back to Polars DataFrame if the original input was Polars
        df = pl.DataFrame(df)

    return df

def null_preprocessing(df):
    if isinstance(df, pl.DataFrame):
        # Convert Polars dataframe to Pandas dataframe
        df = df.to_pandas()
    non_null_columns = [col for col in df.columns if df[col].notnull().all()]
    null_columns = [col for col in df.columns if df[col].isnull().any()]

    for column in null_columns:
        print(column)
        df = linear_regression_imputation(df, column, non_null_columns)
    return df

def replace_boolean_with_numeric(df):
    # 데이터프레임 내의 모든 True를 1로, False를 0으로 바꿈
    df = df.replace({True: 1, False: 0})
    return df

from sklearn.impute import KNNImputer

def knn_imputer_column(df, column_name, n_neighbors=3):
    """
    선택한 열에 대해서만 KNN 기반 결측치 대체를 수행하는 함수

    :param df: 데이터프레임
    :param column_name: 대체하고자 하는 열의 이름
    :param n_neighbors: KNN 알고리즘에서 이웃의 수
    :return: 선택한 열의 결측치가 대체된 시리즈
    """
    # 대상 열만 있는 데이터프레임 생성
    df_subset = df[[column_name]]

    # KNNImputer 객체 생성
    imputer = KNNImputer(n_neighbors=n_neighbors)

    # 결측치 대체
    filled_array = imputer.fit_transform(df_subset)

    # 대체된 값을 시리즈로 변환
    filled_series = pd.Series(filled_array[:, 0], name=column_name, index=df.index)

    return filled_series


import pandas as pd

def merge_columns(df, common_name, col_1, col_2):
    temp_df_1 = df[['case_id', 'num_group1', col_1]].copy()
    temp_df_1.dropna(subset=[col_1], inplace=True)
    temp_df_1.rename(columns={col_1: common_name}, inplace=True)

    temp_df_2 = df[['case_id', 'num_group1', col_2]].copy()
    temp_df_2.dropna(subset=[col_2], inplace=True)
    temp_df_2.rename(columns={col_2: common_name}, inplace=True)

    merged_df = pd.concat([temp_df_1, temp_df_2])
    merged_df.drop_duplicates(subset=['case_id', 'num_group1'], inplace=True)

    df = pd.merge(df, merged_df, how='outer', on=['case_id', 'num_group1'])

    df.drop(columns=col_1, inplace=True)
    df.drop(columns=col_2, inplace=True)

    return df

import pandas as pd

def find_missing_rows(df, column_name):
    """
    데이터프레임의 특정 열의 결측치의 행 번호를 반환하는 함수

    :param df: 데이터프레임
    :param column_name: 결측치를 확인할 열의 이름
    :return: 결측치가 있는 행의 번호 리스트
    """
    missing_rows = df[df[column_name].isnull()].index.tolist()
    return missing_rows

In [3]:
!pip install fastparquet
!pip install pyarrow


# 0. package import

In [4]:
import polars as pl
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import gc
dataPath = "/kaggle/input/home-credit-credit-risk-model-stability/"

# 1. data load

In [5]:
def downcast_df(df):
    # Integer columns downcast
    int_cols = df.select_dtypes(include=['int64', 'int32', 'int16', 'int8']).columns
    for col in int_cols:
        df[col] = df[col].astype('int16')

    # Float columns downcast
    float_cols = df.select_dtypes(include=['float64', 'float32']).columns
    for col in float_cols:
        df[col] = df[col].astype('float32')

    return df

## depth2 - 1/3

In [6]:
import cudf

# Initialize an empty dataframe for concatenation
combined_df = None

# Process files in batches to manage memory usage
for i in range(0,6):
    gc.collect()
    print(i)
    # Read the parquet file using cudf
    df = cudf.read_parquet(f"{dataPath}parquet_files/train/train_credit_bureau_a_2_{i}.parquet")
    
    # Convert the cudf DataFrame to pandas DataFrame
    df_pd = df.to_pandas()
    del df
    # Downcast the pandas DataFrame
    df_downcasted = downcast_df(df_pd)
    del df_pd
    # Concatenate the downcasted pandas DataFrame
    if combined_df is None:
        combined_df = df_downcasted
    else:
        combined_df = pd.concat([combined_df, df_downcasted], axis=0)
        
    # Explicitly delete the temporary DataFrame to free memory
    del df_downcasted
    gc.collect()

# Now you can continue working with the combined DataFrame
train_credit_bureau_a_2_0_5 = combined_df


/opt/conda/lib/python3.10/site-packages/cudf/utils/_numba.py:110: UserWarning: Using CUDA toolkit version (12, 4) with CUDA driver version (12, 2) requires minor version compatibility, which is not yet supported for CUDA driver versions 12.0 and above. It is likely that many cuDF operations will not work in this state. Please install CUDA toolkit version (12, 2) to continue using cuDF.
  warnings.warn(
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcWriteOptions size changed, may indicate binary incompatibility. Expected 72 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcReadOptions size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.NativeFile size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.B

0


get_mempolicy: Operation not permitted


1
2
3
4
5


In [10]:
gc.collect()

960

In [ ]:
# Initialize an empty dataframe for concatenation
combined_df = None

# Process files in batches to manage memory usage
for i in range(12):
    gc.collect()
    print(i)
    df = pl.read_parquet(f"{dataPath}parquet_files/test/test_credit_bureau_a_2_{i}.parquet")
    df_pd = df.to_pandas()
    df_downcasted = downcast_df(df_pd)

    if combined_df is None:
        combined_df = df_downcasted
    else:
        combined_df = pd.concat([combined_df, df_downcasted], axis=0)
        # Explicitly delete the temporary dataframe to free memory
        del df_pd, df_downcasted, df

# Convert the combined dataframe back to Polars dataframe
test_credit_bureau_a_2 = pl.from_pandas(combined_df)


In [ ]:
train_credit_bureau_b_2 = pl.read_parquet(dataPath + "parquet_files/train/train_credit_bureau_b_2.parquet")
test_credit_bureau_b_2 = pl.read_parquet(dataPath + "parquet_files/test/test_credit_bureau_b_2.parquet")

## depth0

In [ ]:
#train
train_basetable = pl.read_parquet(dataPath + "parquet_files/train/train_base.parquet")
train_static = pl.concat(
    [
        pl.read_parquet(dataPath + "parquet_files/train/train_static_0_0.parquet"),
        pl.read_parquet(dataPath + "parquet_files/train/train_static_0_1.parquet"),
    ],
    how="vertical_relaxed",
)
train_static = train_static.to_pandas()
train_static = downcast_df(train_static)
train_static = pl.DataFrame(train_static)

train_static_cb = pl.read_parquet(dataPath + "parquet_files/train/train_static_cb_0.parquet")
train_static_cb = train_static_cb.to_pandas()
train_static_cb = downcast_df(train_static_cb)
train_static_cb = pl.DataFrame(train_static_cb)

#test
test_basetable = pl.read_parquet(dataPath + "parquet_files/test/test_base.parquet")
test_static = pl.concat(
    [
        pl.read_parquet(dataPath + "parquet_files/test/test_static_0_0.parquet"),
        pl.read_parquet(dataPath + "parquet_files/test/test_static_0_1.parquet"),
        pl.read_parquet(dataPath + "parquet_files/test/test_static_0_2.parquet"),
    ],
    how="vertical_relaxed",
)
test_static = test_static.to_pandas()
test_static = downcast_df(test_static)
test_static = pl.DataFrame(test_static)

test_static_cb = pl.read_parquet(dataPath + "parquet_files/test/test_static_cb_0.parquet")
test_static_cb = test_static_cb.to_pandas()
test_static_cb = downcast_df(test_static_cb)


In [ ]:
import gc
gc.collect()

## depth1

In [1]:
#train
train_applprev_1 = pl.concat(
    [
        pl.read_parquet(dataPath + "parquet_files/train/train_base.parquet"),
        pl.read_parquet(dataPath + "train_applprev_1_1.parquet"),
    ],
    how="vertical_relaxed",
)
train_bureau_a_1_0 = pl.read_parquet("parquet_files/train/train_credit_bureau_a_1_0.parquet")
train_bureau_a_1_1 = pl.read_parquet("parquet_files/train/train_credit_bureau_a_1_1.parquet")
train_bureau_a_1_2 = pl.read_parquet("parquet_files/train/train_credit_bureau_a_1_2.parquet")
train_bureau_a_1_3 = pl.read_parquet("parquet_files/train/train_credit_bureau_a_1_3.parquet")
train_bureau_b_1 = pl.read_parquet(dataPath + "/train_credit_bureau_b_1.parquet")
train_debitcard_1 = pl.read_parquet(dataPath + "train_debitcard_1.parquet")
train_deposit_1 = pl.read_parquet(dataPath + "train_deposit_1.parquet")
train_other_1 = pl.read_parquet(dataPath + "train_other_1.parquet")
train_person_1 = pl.read_parquet(dataPath + "train_person_1.parquet")
train_tax_registry_a_1 = pl.read_parquet(dataPath + "train_tax_registry_a_1.parquet")
train_tax_registry_b_1 = pl.read_parquet(dataPath + "train_tax_registry_b_1.parquet")
train_tax_registry_c_1 = pl.read_parquet(dataPath + "train_tax_registry_c_1.parquet")

#test
test_applprev = pl.concat(
    [
        pl.read_parquet(dataPath + "test_applprev_1_0.parquet"),
        pl.read_parquet(dataPath + "test_applprev_1_1.parquet"),
        pl.read_parquet(dataPath + "test_applprev_1_2.parquet"),
    ],
    how="vertical_relaxed",
)

test_bureau_a = pl.concat(
    [
        pl.read_parquet(dataPath + "test_credit_bureau_a_1_0.parquet"),
        pl.read_parquet(dataPath + "test_credit_bureau_a_1_1.parquet"),
        pl.read_parquet(dataPath + "test_credit_bureau_a_1_2.parquet"),
        pl.read_parquet(dataPath + "test_credit_bureau_a_1_3.parquet"),
        pl.read_parquet(dataPath + "test_credit_bureau_a_1_4.parquet"),
    ],
    how="vertical_relaxed",
)

test_bureau_b_1 = pl.read_parquet(dataPath + "test_credit_bureau_b_1.parquet")

test_debitcard_1 = pl.read_parquet(dataPath + "test_debitcard_1.parquet")
test_deposit_1 = pl.read_parquet(dataPath + "test_deposit_1.parquet")
test_other_1 = pl.read_parquet(dataPath + "test_other_1.parquet")
test_person_1 = pl.read_parquet(dataPath + "test_person_1.parquet")

test_tax_registry_a_1 = pl.read_parquet(dataPath + "test_tax_registry_a_1.parquet")
test_tax_registry_b_1 = pl.read_parquet(dataPath + "test_tax_registry_b_1.parquet")
test_tax_registry_c_1 = pl.read_parquet(dataPath + "test_tax_registry_c_1.parquet")

NameError: name 'pl' is not defined

# 2. data preprocessing

## depth0 preprocessing

### static

In [ ]:
# 모든 값이 null인 열 찾기
null_columns = find_all_null_columns(test_static)
train_static = train_static.drop(null_columns)
test_static = test_static.drop(null_columns)

In [ ]:
train_static, train_static_columns_to_drop = train_static_depth0_preprocess(train_static)

In [ ]:
gc.collect()

In [ ]:
test_static = test_static_depth0_preprocess(test_static, train_static_columns_to_drop)

In [ ]:
test_static = test_train_col_equ(train_static,test_static)

### static_cb

In [ ]:
test_static_cb = pl.from_pandas(test_static_cb)

In [ ]:
# 모든 값이 null인 열 찾기
null_columns = find_all_null_columns(test_static_cb)
train_static_cb = train_static_cb.drop(null_columns)
test_static_cb = test_static_cb.drop(null_columns)

In [ ]:
train_static_cb, train_static_cb_columns_to_drop = train_static_cb_depth0_preprocess(train_static_cb)

In [ ]:
test_static_cb = test_static_cb_depth0_preprocess(test_static_cb, train_static_cb_columns_to_drop)

In [ ]:
test_static_cb = test_train_col_equ(train_static_cb,test_static_cb)

In [ ]:
test_static_cb = test_static_cb.to_pandas()
train_static_cb = train_static_cb.to_pandas()

test_static = test_static.to_pandas()
train_static = train_static.to_pandas()

## depth 1

## depth2

applprev_depth2는 제외

- credit_bureau_a_2

In [ ]:
# 모든 값이 null인 열 찾기
null_columns = find_all_null_columns(test_credit_bureau_a_2)
train_credit_bureau_a_2_0 = train_credit_bureau_a_2.drop(null_columns)
test_credit_bureau_a_2_0 = test_credit_bureau_a_2.drop(null_columns)

In [ ]:
def num_group_by_0(df):
    # 각 그룹의 평균 계산
    df_mean = df.groupby('case_id').mean()
    df_mean.drop(['num_group1','num_group2'])
    return df_mean

In [ ]:
train_credit_bureau_a_2_0_mean = num_group_by_0(train_credit_bureau_a_2_0)
train_credit_bureau_a_2_0_mean

In [ ]:
test_credit_bureau_a_2_0_mean = num_group_by_0(test_credit_bureau_a_2_0)
test_credit_bureau_a_2_0_mean.tail(10)

In [ ]:
gc.collect()

In [ ]:
train_credit_bureau_a_2_0_mean, train_credit_bureau_a_columns_to_drop = train_credit_applprev_depth2_preprocess(train_credit_bureau_a_2_0_mean)

In [ ]:
test_credit_bureau_a_2_0_mean = test_credit_applprev_depth2_preprocess(test_credit_bureau_a_2_0_mean, train_credit_bureau_a_columns_to_drop)

In [ ]:
test_credit_bureau_a_2_0_mean['case_id'].tail(10)

## (1) data set 합치기

### depth0

In [ ]:
train_basetable_temp = train_basetable.to_pandas()
test_basetable_temp = test_basetable.to_pandas()

In [ ]:
# object 타입인 열들을 찾음
object_columns_test = test_basetable_temp.select_dtypes(include=['object']).columns
object_columns_train = train_basetable_temp.select_dtypes(include=['object']).columns

# object 타입인 열들을 제거
test_basetable_temp = test_basetable_temp.drop(columns=object_columns_test)
train_basetable_temp = train_basetable_temp.drop(columns=object_columns_train)

In [ ]:
train_static = pl.DataFrame(train_static)
train_static_cb = pl.DataFrame(train_static_cb)
test_static = pl.DataFrame(test_static)
test_static_cb = pl.DataFrame(test_static_cb)

In [ ]:
train_static = train_static.to_pandas()
train_static_cb = train_static_cb.to_pandas()
test_static = test_static.to_pandas()
test_static_cb = test_static_cb.to_pandas()

In [ ]:
depth_0_train = pd.merge(train_static, train_static_cb, on='case_id', how='inner')
depth_0_test = pd.merge(test_static, test_static_cb, on='case_id', how='left')

In [ ]:
base_depth_0_train = pd.merge(train_basetable_temp, depth_0_train, on='case_id', how='inner')
base_depth_0_test = pd.merge(test_basetable_temp, depth_0_test, on='case_id', how='left')

In [ ]:
base_depth_0_train

### depth2

In [ ]:
depth_2_train = train_credit_bureau_a_2_0_mean.to_pandas()
depth_2_test = test_credit_bureau_a_2_0_mean.to_pandas()


In [ ]:
base_depth_0_depth_2_train = pd.merge(base_depth_0_train, depth_2_train, on='case_id', how='inner')
base_depth_0_depth_2_test = pd.merge(base_depth_0_test, depth_2_test, on='case_id', how='left')


In [ ]:
base_depth_0_depth_2_train

# 3. data train

In [ ]:
import polars as pl
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
train_data = base_depth_0_depth_2_train
test_data = base_depth_0_depth_2_test

In [ ]:
train_data = pl.DataFrame(train_data)
test_data = pl.DataFrame(test_data)
train_data = train_data.to_pandas()
test_data = test_data.to_pandas()

In [ ]:
# 훈련 및 테스트 데이터로 분할
train_df, test_df = train_test_split(train_data, test_size=0.2, random_state=42)
train_df = pl.DataFrame(train_df)
test_data = pl.DataFrame(test_data)


In [ ]:
gc.collect()

In [ ]:
del depth_2_train
del depth_2_test

In [ ]:
def convert_to_dataframe(data):
    # 데이터가 판다스 DataFrame 타입인지 확인
    if isinstance(data, pd.DataFrame):
        # 이미 판다스 DataFrame이면 그대로 반환
        return data
    else:
        # 판다스 DataFrame으로 변환
            converted_data = pd.DataFrame(data)
            return converted_data


In [ ]:
test_data = test_data.drop('case_id')
X_train = train_df.drop(['case_id', 'target'])
y_train = train_df['target']
X_test = test_df.drop(columns=['case_id', 'target'])
y_test = test_df['target']

In [ ]:
test_data = convert_to_dataframe(test_data)
X_train = convert_to_dataframe(X_train)
y_train = convert_to_dataframe(y_train)
X_test = convert_to_dataframe(X_test)
y_test = convert_to_dataframe(y_test)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import lightgbm as lgb

# K-fold 교차 검증 설정
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# PCA 설정
pca = PCA(n_components=2)

# K-fold 교차 검증을 통한 모델 평가
accuracy_scores = []

for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # PCA를 훈련 데이터에 적용하여 차원 축소
    X_train_pca = pca.fit_transform(X_train_fold)
    X_val_pca = pca.transform(X_val_fold)

    # LightGBM 모델 훈련
    model_lgb = lgb.LGBMClassifier()
    model_lgb.fit(X_train_pca, y_train_fold)

    # 검증 데이터에 대한 예측 수행
    y_val_pred = model_lgb.predict(X_val_pca)

    # 정확도 계산
    accuracy = accuracy_score(y_val_fold, y_val_pred)
    accuracy_scores.append(accuracy)

# K-fold 교차 검증 결과 출력
print(f"Accuracy scores for each fold: {accuracy_scores}")
print(f"Mean accuracy: {np.mean(accuracy_scores)}")


In [ ]:
test_data = null_preprocessing(test_data)

In [ ]:
test_data = pca.transform(test_data)


In [ ]:
test_y = model_lgb.predict(test_data)

In [ ]:
dataPath = "/kaggle/input/home-credit-credit-risk-model-stability/"

In [ ]:
submission = pd.read_csv(dataPath + 'sample_submission.csv')
submission['TARGET'] = test_y
submission.to_csv('submission.csv', index=False)